In [1]:
params_appliance = {
    'kettle': {
        'windowlength': 599,
        'on_power_threshold': 2000,
        'max_on_power': 3998,
        'mean': 700,
        'std': 1000,
        's2s_length': 128,
        'houses': [1, 2],
        'channels': [10, 8],
        'train_build': [1],
        'test_build': 2,
    },
    'microwave': {
        'windowlength': 599,
        'on_power_threshold': 200,
        'max_on_power': 3969,
        'mean': 500,
        'std': 800,
        's2s_length': 128,
        'houses': [1, 2],
        'channels': [13, 15],
        'train_build': [1],
        'test_build': 2,
    },
    'fridge': {
        'windowlength': 599,
        'on_power_threshold': 50,
        'max_on_power': 3323,
        'mean': 200,
        'std': 400,
        's2s_length': 512,
        'houses': [1, 2],
        'channels': [12, 14],
        'train_build': [1],
        'test_build': 2,
    },
    'dishwasher': {
        'windowlength': 599,
        'on_power_threshold': 10,
        'max_on_power': 3964,
        'mean': 700,
        'std': 1000,
        's2s_length': 1536,
        'houses': [1, 2],
        'channels': [6, 13],
        'train_build': [1],
        'test_build': 2,
    },
    'washingmachine': {
        'windowlength': 599,
        'on_power_threshold': 20,
        'max_on_power': 3999,
        'mean': 400,
        'std': 700,
        's2s_length': 2000,
        'houses': [1, 2],
        'channels': [5, 12],
        'train_build': [1],
        'test_build': 2,
    }
}

DATA_DIRECTORY = 'dataset/ukdale/'
SAVE_PATH = 'UKDALE/kettle/'
AGG_MEAN = 522
AGG_STD = 814


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import argparse



DATA_DIRECTORY = 'dataset/ukdale/'
SAVE_PATH = 'UKDALE/kettle/'
AGG_MEAN = 522
AGG_STD = 814

# def load_dataframe(data_dir, house, channel, col_names=None):
#     """
#     Load a dataframe from the UKDALE dataset.
#     :param data_dir: Directory where the UKDALE data is stored.
#     :param house: House number to load data from.
#     :param channel: Channel number to load data from.
#     :param col_names: Optional list of column names for the dataframe.
#     :return: DataFrame containing the loaded data.
#     """
#     file_path = f"{data_dir}house_{house}/channel_{channel}.dat"
#     df = pd.read_csv(file_path, header=None, names=col_names)
#     return df

def load_dataframe(directory, building, channel, col_names=['time', 'data'], nrows=None):
    df = pd.read_table(directory + 'house_' + str(building) + '/' + 'channel_' +
                       str(channel) + '.dat',
                       sep="\s+",
                       nrows=nrows,
                       usecols=[0, 1],
                       names=col_names,
                       dtype={'time': str},
                       )
    return df



#def get_arguments():
#   parser = argparse.ArgumentParser(description='sequence to point learning \
#                                     example for NILM')
#    parser.add_argument('--data_dir', type=str, default=DATA_DIRECTORY,
#                          help='The directory containing the UKDALE data')
#    parser.add_argument('--appliance_name', type=str, default='kettle',
#                          help='which appliance you want to train: kettle,\
#                          microwave,fridge,dishwasher,washingmachine')
#   parser.add_argument('--aggregate_mean',type=int,default=AGG_MEAN,
#                        help='Mean value of aggregated reading (mains)')
#    parser.add_argument('--aggregate_std',type=int,default=AGG_STD,
#                        help='Std value of aggregated reading (mains)')
#    parser.add_argument('--save_path', type=str, default=SAVE_PATH,
#                          help='The directory to store the training data')
#    return parser.parse_args()
class get_arguments:
    def __init__(self):
        self.data_dir = DATA_DIRECTORY
        self.appliance_name = 'kettle'
        self.aggregate_mean = AGG_MEAN
        self.aggregate_std = AGG_STD
        self.save_path = SAVE_PATH

args = get_arguments()
appliance_name = args.appliance_name
print(appliance_name)


def main():

    start_time = time.time()
    sample_seconds = 8
    training_building_percent = 95
    validation_percent = 13
    nrows = None
    debug = False

    train = pd.DataFrame(columns=['aggregate', appliance_name])

    for h in params_appliance[appliance_name]['houses']:
        print('    ' + args.data_dir + 'house_' + str(h) + '/'
              + 'channel_' +
              str(params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)]) +
              '.dat')

        mains_df = load_dataframe(args.data_dir, h, 1)
        app_df = load_dataframe(args.data_dir,
                                h,
                                params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)],
                                col_names=['time', appliance_name]
                                )

        mains_df['time'] = pd.to_datetime(mains_df['time'], unit='s')
        mains_df.set_index('time', inplace=True)
        mains_df.columns = ['aggregate']
        #resample = mains_df.resample(str(sample_seconds) + 'S').mean()
        mains_df.reset_index(inplace=True)

        if debug:
            print("    mains_df:")
            print(mains_df.head())
            plt.plot(mains_df['time'], mains_df['aggregate'])
            plt.show()

        # Appliance
        app_df['time'] = pd.to_datetime(app_df['time'], unit='s')

        if debug:
            print("app_df:")
            print(app_df.head())
            plt.plot(app_df['time'], app_df[appliance_name])
            plt.show()

        # the timestamps of mains and appliance are not the same, we need to align them
        # 1. join the aggragte and appliance dataframes;
        # 2. interpolate the missing values;
        mains_df.set_index('time', inplace=True)
        app_df.set_index('time', inplace=True)

        df_align = mains_df.join(app_df, how='outer'). \
            resample(str(sample_seconds) + 'S').mean().fillna(method='backfill', limit=1)
        df_align = df_align.dropna()

        df_align.reset_index(inplace=True)

        del mains_df, app_df, df_align['time']

        if debug:
            # plot the dtaset
            print("df_align:")
            print(df_align.head())
            plt.plot(df_align['aggregate'].values)
            plt.plot(df_align[appliance_name].values)
            plt.show()

        # Normilization ----------------------------------------------------------------------------------------------
        mean = params_appliance[appliance_name]['mean']
        std = params_appliance[appliance_name]['std']

        df_align['aggregate'] = (df_align['aggregate'] - args.aggregate_mean) / args.aggregate_std
        df_align[appliance_name] = (df_align[appliance_name] - mean) / std

        if h == params_appliance[appliance_name]['test_build']:
            # Test CSV
            df_align.to_csv(args.save_path + appliance_name + '_test_.csv', mode='a', index=False, header=False)
            print("    Size of test set is {:.4f} M rows.".format(len(df_align) / 10 ** 6))
            continue

        train = pd.concat([train, df_align], ignore_index=True)
        del df_align

    # Crop dataset
    if training_building_percent is not 0:
        train.drop(train.index[-int((len(train)/100)*training_building_percent):], inplace=True)


    # Validation CSV
    val_len = int((len(train)/100)*validation_percent)
    val = train.tail(val_len)
    val.reset_index(drop=True, inplace=True)
    train.drop(train.index[-val_len:], inplace=True)
    # Validation CSV
    val.to_csv(args.save_path + appliance_name + '_validation_' + '.csv', mode='a', index=False, header=False)

    # Training CSV
    train.to_csv(args.save_path + appliance_name + '_training_.csv', mode='a', index=False, header=False)

    print("    Size of total training set is {:.4f} M rows.".format(len(train) / 10 ** 6))
    print("    Size of total validation set is {:.4f} M rows.".format(len(val) / 10 ** 6))
    del train, val


    print("\nPlease find files in: " + args.save_path)
    print("Total elapsed time: {:.2f} min.".format((time.time() - start_time) / 60))


if __name__ == '__main__':
    main()

<>:151: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:151: SyntaxWarning: "is not" with a literal. Did you mean "!="?


KeyboardInterrupt: 

In [4]:
start_time = time.time()
appliance_name = 'kettle'
print(appliance_name)

# UK-DALE path
path = 'dataset/ukdale/'
save_path = 'UKDALE/kettle/'

aggregate_mean = 522
aggregate_std = 814

nrows = 10**5



def load(path, building, appliance, channel, nrows=None):
    # load csv
    file_name = path + 'house_' + str(building) + '/' + 'channel_' + str(channel) + '.dat'
    single_csv = pd.read_csv(file_name,
                             sep=' ',
                             #header=0,
                             names=['time', appliance],
                             dtype={'time': str, "appliance": int},
                             #parse_dates=['time'],
                             #date_parser=pd.to_datetime,
                             nrows=nrows,
                             usecols=[0, 1],
                             engine='python'
                             )
    return single_csv




print("Starting creating testset...")

for h in params_appliance[appliance_name]['houses']:

    print(path + 'house_' + str(h) + '/'
          + 'channel_' +
          str(params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)]) +
          '.dat')

    agg_df = load(path,
                  h,
                  appliance_name,
                  1,
                  nrows=nrows,
                  )

    df = load(path,
              h,
              appliance_name,
              params_appliance[appliance_name]['channels'][params_appliance[appliance_name]['houses'].index(h)],
              nrows=nrows,
              )

    #for i in range(100):
    #    print(int(df['time'][i]) - int(agg_df['time'][i]))

    # Time conversion
    print(df.head())
    print(agg_df.head())
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    agg_df['time'] = pd.to_datetime(agg_df['time'], unit='ms')
    print(agg_df.head())
    print(df.head())

    df['aggregate'] = agg_df[appliance_name]
    cols = df.columns.tolist()
    del cols[0]
    cols = cols[-1:] + cols[:-1]
    df = df[cols]

    print(df.head())


    # Re-sampling
    ind = pd.date_range(0,  periods=df.shape[0], freq='6S')
    df.set_index(ind, inplace=True, drop=True)
    resample = df.resample('8S')
    df = resample.mean()

    print(df.head())

    # Normalization
    df['aggregate'] = (df['aggregate'] - aggregate_mean) / aggregate_std
    df[appliance_name] = \
        (df[appliance_name] - params_appliance[appliance_name]['mean']) / params_appliance[appliance_name]['std']

    # Save
    df.to_csv(save_path + appliance_name + '_test_' + 'uk-dale_' + 'H' + str(h) + '.csv', index=False)

    print("Size of test set is {:.3f} M rows (House {:d})."
          .format(df.shape[0] / 10 ** 6, h))

    del df


print("\nNormalization parameters: ")
print("Mean and standard deviation values USED for AGGREGATE are:")
print("    Mean = {:d}, STD = {:d}".format(aggregate_mean, aggregate_std))

print('Mean and standard deviation values USED for ' + appliance_name + ' are:')
print("    Mean = {:d}, STD = {:d}"
      .format(params_appliance[appliance_name]['mean'], params_appliance[appliance_name]['std']))

print("\nPlease find files in: " + save_path)
tot = int(int(time.time() - start_time) / 60)
print("\nTotal elapsed time: " + str(tot) + ' min')





kettle
Starting creating testset...
dataset/ukdale/house_1/channel_10.dat
         time  kettle
0  1352500098       1
1  1352500104       1
2  1352500110       1
3  1352500116       1
4  1352500122       1
         time  kettle
0  1352500095     599
1  1352500101     582
2  1352500107     600
3  1352500113     586
4  1352500120     596


C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:64: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['time'] = pd.to_datetime(df['time'], unit='ms')
C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:65: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  agg_df['time'] = pd.to_datetime(agg_df['time'], unit='ms')


                     time  kettle
0 1970-01-16 15:41:40.095     599
1 1970-01-16 15:41:40.101     582
2 1970-01-16 15:41:40.107     600
3 1970-01-16 15:41:40.113     586
4 1970-01-16 15:41:40.120     596
                     time  kettle
0 1970-01-16 15:41:40.098       1
1 1970-01-16 15:41:40.104       1
2 1970-01-16 15:41:40.110       1
3 1970-01-16 15:41:40.116       1
4 1970-01-16 15:41:40.122       1
   aggregate  kettle
0        599       1
1        582       1
2        600       1
3        586       1
4        596       1
                     aggregate  kettle
1970-01-01 00:00:00      590.5     1.0
1970-01-01 00:00:08      600.0     1.0
1970-01-01 00:00:16      586.0     1.0
1970-01-01 00:00:24      588.5     1.0
1970-01-01 00:00:32      597.0     1.0


C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:79: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  ind = pd.date_range(0,  periods=df.shape[0], freq='6S')
C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:81: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  resample = df.resample('8S')


Size of test set is 0.075 M rows (House 1).
dataset/ukdale/house_2/channel_8.dat
         time  kettle
0  1361116822       0
1  1361116825       0
2  1361116831       0
3  1361116837       0
4  1361116843       0
         time  kettle
0  1361117854     340
1  1361117860     341
2  1361117866     347
3  1361117872     350
4  1361117878     342


C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:64: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['time'] = pd.to_datetime(df['time'], unit='ms')
C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:65: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  agg_df['time'] = pd.to_datetime(agg_df['time'], unit='ms')


                     time  kettle
0 1970-01-16 18:05:17.854     340
1 1970-01-16 18:05:17.860     341
2 1970-01-16 18:05:17.866     347
3 1970-01-16 18:05:17.872     350
4 1970-01-16 18:05:17.878     342
                     time  kettle
0 1970-01-16 18:05:16.822       0
1 1970-01-16 18:05:16.825       0
2 1970-01-16 18:05:16.831       0
3 1970-01-16 18:05:16.837       0
4 1970-01-16 18:05:16.843       0
   aggregate  kettle
0        340       0
1        341       0
2        347       0
3        350       0
4        342       0
                     aggregate  kettle
1970-01-01 00:00:00      340.5     0.0
1970-01-01 00:00:08      347.0     0.0
1970-01-01 00:00:16      350.0     0.0
1970-01-01 00:00:24      341.5     0.0
1970-01-01 00:00:32      343.0     0.0


C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:79: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  ind = pd.date_range(0,  periods=df.shape[0], freq='6S')
C:\Users\HP\AppData\Local\Temp\ipykernel_14180\1025023502.py:81: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  resample = df.resample('8S')


Size of test set is 0.075 M rows (House 2).

Normalization parameters: 
Mean and standard deviation values USED for AGGREGATE are:
    Mean = 522, STD = 814
Mean and standard deviation values USED for kettle are:
    Mean = 700, STD = 1000

Please find files in: UKDALE/kettle/

Total elapsed time: 0 min


In [2]:
import logging
import time
import socket

log_file_name = '{}.log'.format(time.strftime("%Y-%m-%d-%H:%M:%S").replace(':','-'))

with open(log_file_name, 'w'):
    pass

logFormatter = logging.Formatter("%(asctime)s [%(levelname)-5.5s]  %(message)s")
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.DEBUG)
fileHandler = logging.FileHandler("{0}".format(log_file_name))
fileHandler.setFormatter(logFormatter)
rootLogger.addHandler(fileHandler)

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
rootLogger.addHandler(consoleHandler)


def log(string, level='info'):

    if level == 'info':
        rootLogger.info(string)
    elif level == 'debug':
        rootLogger.debug(string)
    elif level == 'warning':
        rootLogger.warning(string)

log('Parameters: ')
machine_name = socket.gethostname()
log('Machine name: ' + machine_name)

2025-06-20 02:57:53,421 [INFO ]  Parameters: 
2025-06-20 02:57:53,422 [INFO ]  Machine name: DESKTOP-N5CITIQ


In [12]:
from keras.models import Model
from keras.layers import Dense, Conv2D, Flatten, Reshape
from keras.utils import plot_model
import numpy as np
import keras.backend as K
import os
import tensorflow as tf
import h5py
import argparse


def get_model(appliance, input_tensor, window_length, transfer_dense=False, transfer_cnn=False,
              cnn='kettle', n_dense=1, pretrainedmodel_dir='saved_models/kettle_best_model'):

    reshape = Reshape((-1, window_length, 1),
                      )(input_tensor)

    cnn1 = Conv2D(filters=30,
                  kernel_size=(10, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  )(reshape)

    cnn2 = Conv2D(filters=30,
                  kernel_size=(8, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  )(cnn1)

    cnn3 = Conv2D(filters=40,
                  kernel_size=(6, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  )(cnn2)

    cnn4 = Conv2D(filters=50,
                  kernel_size=(5, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  )(cnn3)

    cnn5 = Conv2D(filters=50,
                  kernel_size=(5, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  )(cnn4)

    flat = Flatten(name='flatten')(cnn5)

    d = Dense(1024, activation='relu', name='dense')(flat)

    if n_dense == 1:
        label = d
    elif n_dense == 2:
        d1 = Dense(1024, activation='relu', name='dense1')(d)
        label = d1
    elif n_dense == 3:
        d1 = Dense(1024, activation='relu', name='dense1')(d)
        d2 = Dense(1024, activation='relu', name='dense2')(d1)
        label = d2

    d_out = Dense(1, activation='linear', name='output')(label)

    model = Model(inputs=input_tensor, outputs=d_out)

    session = K.get_session()

    if transfer_dense:
        log("Transfer learning...")
        log("...loading an entire pre-trained model")
        weights_loader(model, pretrainedmodel_dir+'/cnn_s2p_' + appliance + '_pointnet_model')
        model_def = model
    elif transfer_cnn and not transfer_dense:
        log("Transfer learning...")
        log('...loading a ' + appliance + ' pre-trained-cnn')
        cnn_weights_loader(model, cnn, pretrainedmodel_dir)
        model_def = model
        for idx, layer1 in enumerate(model_def.layers):
            if hasattr(layer1, 'kernel_initializer') and 'conv2d' not in layer1.name and 'cnn' not in layer1.name:
                log('Re-initialize: {}'.format(layer1.name))
                layer1.kernel.initializer.run(session=session)

    elif not transfer_dense and not transfer_cnn:
        log("Standard training...")
        log("...creating a new model.")
        model_def = model
    else:
        raise argparse.ArgumentTypeError('Model selection error.')

    # Printing, logging and plotting the model
    model_def.summary()
    #plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')
    #plot_model(model_def, to_file='model_def.png', show_shapes=True, show_layer_names=True, rankdir='TB')

    # Adding network structure to both the log file and output terminal
    files = [x for x in os.listdir('./') if x.endswith(".log")]
    with open(max(files, key=os.path.getctime), 'a') as fh:
        # Pass the file handle in as a lambda function to make it callable
        model_def.summary(print_fn=lambda x: fh.write(x + '\n'))

    # Check weights slice
    for v in tf.compat.v1.trainable_variables():
        if v.name == 'conv2d_1/kernel:0':
            cnn1_weights = session.run(v)
    return model_def, cnn1_weights


def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))


def cnn_weights_loader(model_to_fill, cnn_appliance, pretrainedmodel_dir):
    log('Loading cnn weights from ' + cnn_appliance)    
    weights_path = pretrainedmodel_dir+'/cnn_s2p_' + cnn_appliance + '_pointnet_model' + '_weights.h5'
    if not os.path.exists(weights_path):
        print('The directory does not exist or you do not have the files for trained model')
        
    f = h5py.File(weights_path, 'r')
    log(f.visititems(print_attrs))
    layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
    for name in layer_names:
        if 'conv2d_' in name or 'cnn' in name:
            g = f[name]
            weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
            if len(weight_names):
                weight_values = [g[weight_name] for weight_name in weight_names]

            model_to_fill.layers[int(name[-1])+1].set_weights(weight_values)
            log('Loaded cnn layer: {}'.format(name))

    f.close()
    print('Model loaded.')


def weights_loader(model, path):
    log('Loading cnn weights from ' + path)
    model.load_weights(path + '_weights.h5')







In [6]:
class ChunkDoubleSourceSlider(object):
    def __init__(self, filename, batchsize, chunksize, shuffle, offset, crop=None, header=0, ram_threshold=5*10**5):

        self.filename = filename
        self.batchsize = batchsize
        self.chunksize = chunksize
        self.shuffle = shuffle
        self.offset = offset
        self.header = header
        self.crop = crop
        self.ram = ram_threshold

    def check_lenght(self):
        # check the csv size
        check_cvs = pd.read_csv(self.filename,
                                nrows=self.crop,
                                chunksize=10 ** 3,
                                header=self.header
                                )

        t_size = 0

        for chunk in check_cvs:
            size = chunk.shape[0]
            t_size += size
            del chunk
        log('Size of the dataset is {:.3f} M rows.'.format(t_size/10 ** 6))
        if t_size > self.ram:  # IF dataset is too large for memory
            log('It is too large to fit in memory so it will be loaded in chunkes of size {:}.'.format(self.chunksize))
        else:
            log('This size can fit the memory so it will load entirely')

        return t_size

    def feed_chunk(self):

        try:
            total_size
        except NameError:
            #global total_size
            total_size = ChunkDoubleSourceSlider.check_lenght(self)

        if total_size > self.ram:  # IF dataset is too large for memory

            # LOAD data from csv
            data_frame = pd.read_csv(self.filename,
                                     nrows=self.crop,
                                     chunksize=self.chunksize,
                                     header=self.header
                                     )

            # iterations over csv file
            for chunk in data_frame:

                np_array = np.array(chunk)
                inputs, targets = np_array[:, 0], np_array[:, 1]

                """
                if len(inputs) < self.batchsize:
                    while len(inputs) == self.batchsize:
                        inputs = np.append(inputs, 0)
                       targets = np.append(targets, 0)
                """

                max_batchsize = inputs.size - 2 * self.offset
                if self.batchsize < 0:
                    self.batchsize = max_batchsize

                # define indices and shuffle them if necessary
                indices = np.arange(max_batchsize)
                if self.shuffle:
                    np.random.shuffle(indices)

                # providing sliding windows:
                for start_idx in range(0, max_batchsize, self.batchsize):

                    excerpt = indices[start_idx:start_idx + self.batchsize]

                    inp = np.array([inputs[idx:idx + 2 * self.offset + 1] for idx in excerpt])
                    tar = targets[excerpt + self.offset].reshape(-1, 1)

                    yield inp, tar

        else:  # IF dataset can fit the memory

            # LOAD data from csv
            data_frame = pd.read_csv(self.filename,
                                     nrows=self.crop,
                                     header=self.header
                                     )

            np_array = np.array(data_frame)
            inputs, targets = np_array[:, 0], np_array[:, 1]


            max_batchsize = inputs.size - 2 * self.offset
            if self.batchsize < 0:
                    self.batchsize = max_batchsize

            # define indices and shuffle them if necessary
            indices = np.arange(max_batchsize)
            if self.shuffle:
                    np.random.shuffle(indices)

            # providing sliding windows:
            for start_idx in range(0, max_batchsize, self.batchsize):
                excerpt = indices[start_idx:start_idx + self.batchsize]

                inp = np.array([inputs[idx:idx + 2 * self.offset + 1] for idx in excerpt])
                tar = targets[excerpt + self.offset].reshape(-1, 1)

                yield inp, tar


class ChunkDoubleSourceSlider2(object):
    def __init__(self, filename, batchsize, chunksize, shuffle, offset, crop=None, header=0, ram_threshold=5 * 10 ** 5):

        self.filename = filename
        self.batchsize = batchsize
        self.chunksize = chunksize
        self.shuffle = shuffle
        self.offset = offset
        self.header = header
        self.crop = crop
        self.ram = ram_threshold
        self.total_size = 0

    def check_length(self):
        # check the csv size
        check_cvs = pd.read_csv(self.filename,
                                nrows=self.crop,
                                chunksize=10 ** 3,
                                header=self.header
                                )

        for chunk in check_cvs:
            size = chunk.shape[0]
            self.total_size += size
            del chunk
        log('Size of the dataset is {:.3f} M rows.'.format(self.total_size / 10 ** 6))
        if self.total_size > self.ram:  # IF dataset is too large for memory
            log('It is too large to fit in memory so it will be loaded in chunkes of size {:}.'.format(self.chunksize))
        else:
            log('This size can fit the memory so it will load entirely')

    def feed_chunk(self):

        if self.total_size == 0:
            ChunkDoubleSourceSlider2.check_length(self)

        if self.total_size > self.ram:  # IF dataset is too large for memory

            # LOAD data from csv
            data_frame = pd.read_csv(self.filename,
                                     nrows=self.crop,
                                     chunksize=self.chunksize,
                                     header=self.header
                                     )

            skip_idx = np.arange(self.total_size/self.chunksize)
            if self.shuffle:
                np.random.shuffle(skip_idx)

            log(str(skip_idx), 'debug')

            for i in skip_idx:

                log('index: ' + str(i), 'debug')

                # Read the data
                data = pd.read_csv(self.filename,
                                   nrows=self.chunksize,
                                   skiprows=int(i)*self.chunksize,
                                   header=self.header)

                np_array = np.array(data)
                inputs, targets = np_array[:, 0], np_array[:, 1]

                max_batchsize = inputs.size - 2 * self.offset
                if self.batchsize < 0:
                    self.batchsize = max_batchsize

                # define indices and shuffle them if necessary
                indices = np.arange(max_batchsize)
                if self.shuffle:
                    np.random.shuffle(indices)

                # providing sliding windows:
                for start_idx in range(0, max_batchsize, self.batchsize):
                    excerpt = indices[start_idx:start_idx + self.batchsize]

                    inp = np.array([inputs[idx:idx + 2 * self.offset + 1] for idx in excerpt])
                    tar = targets[excerpt + self.offset].reshape(-1, 1)

                    yield inp, tar

        else:  # IF dataset can fit the memory

            # LOAD data from csv
            data_frame = pd.read_csv(self.filename,
                                     nrows=self.crop,
                                     header=self.header
                                     )

            np_array = np.array(data_frame)
            inputs, targets = np_array[:, 0], np_array[:, 1]

            max_batchsize = inputs.size - 2 * self.offset
            if self.batchsize < 0:
                self.batchsize = max_batchsize

            # define indices and shuffle them if necessary
            indices = np.arange(max_batchsize)
            if self.shuffle:
                np.random.shuffle(indices)

            # providing sliding windows:
            for start_idx in range(0, max_batchsize, self.batchsize):
                excerpt = indices[start_idx:start_idx + self.batchsize]

                inp = np.array([inputs[idx:idx + 2 * self.offset + 1] for idx in excerpt])
                tar = targets[excerpt + self.offset].reshape(-1, 1)

                yield inp, tar



    def get_all_data(self):
        """
        Loads the entire dataset from the CSV file and returns X, y numpy arrays.
        Assumes the last column is the target.
        """
        import pandas as pd
        data = pd.read_csv(self.filename, header=self.header)
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values.reshape(-1, 1)
        return X, y


class DoubleSourceProvider2(object):

    def __init__(self, batchsize, shuffle, offset):

        self.batchsize = batchsize
        self.shuffle = shuffle
        self.offset = offset

    def feed(self, inputs, targets):

        assert len(inputs) == len(targets)

        inputs = inputs.flatten()
        targets = targets.flatten()

        max_batchsize = inputs.size - 2 * self.offset

        if self.batchsize == -1:
            self.batchsize = len(inputs)

        indices = np.arange(max_batchsize)
        if self.shuffle:
            np.random.shuffle(indices)

        for start_idx in range(0, max_batchsize, self.batchsize):
            excerpt = indices[start_idx:start_idx + self.batchsize]

            yield np.array([inputs[idx:idx + 2 * self.offset + 1] for idx in excerpt]),\
                  targets[excerpt + self.offset].reshape(-1, 1)


class DoubleSourceProvider3(object):

    def __init__(self, nofWindows, offset):

        self.nofWindows = nofWindows
        self.offset = offset

    def feed(self, inputs):

        inputs = inputs.flatten()
        max_nofw = inputs.size - 2 * self.offset

        if self.nofWindows < 0:
            self.nofWindows = max_nofw

        indices = np.arange(max_nofw, dtype=int)

        # providing sliding windows:
        for start_idx in range(0, max_nofw, self.nofWindows):
            excerpt = indices[start_idx:start_idx + self.nofWindows]

            inp = np.array([inputs[idx:idx + 2 * self.offset + 1] for idx in excerpt])

            yield inp



In [7]:



def dict_to_one(dp_dict={}):

    """ Input a dictionary, return a dictionary that all items are
    set to one, use for disable dropout, drop-connect layer and so on.

    Parameters
    ----------
    dp_dict : dictionary keeping probabilities date
    """
    return {x: 1 for x in dp_dict}


def modelsaver(network, path, epoch_identifier=None):

    if epoch_identifier:
        ifile = path + '_' + str(epoch_identifier)
    else:
        ifile = path

    network.save(ifile + '.h5')
    network.save_weights(ifile + '_weights' + '.h5')


def customfit(sess,
              network,
              cost,
              train_op,
              tra_provider,
              x,
              y_,
              acc=None,
              n_epoch=50,
              print_freq=1,
              val_provider=None,
              save_model=-1,
              tra_kwag=None,
              val_kwag=None,
              save_path=None,
              epoch_identifier=None,
              earlystopping=True,
              min_epoch=1,
              patience=10):
    """
        Traing a given network by the given cost function, dataset, n_epoch etc.

        Parameters
        ----------
        sess : TensorFlow session
            sess = tf.InteractiveSession()
        network : a TensorLayer layer
            the network will be trained
        train_op : a TensorFlow optimizer
            like tf.train.AdamOptimizer
        x : placeholder
            for inputs
        y_ : placeholder
            for targets
        acc : the TensorFlow expression of accuracy (or other metric) or None
            if None, would not display the metric
        batch_size : int
            batch size for training and evaluating
        n_epoch : int
            the number of training epochs
        print_freq : int
            display the training information every ``print_freq`` epochs
        X_val : numpy array or None
            the input of validation data
        y_val : numpy array or None
            the target of validation data
        eval_train : boolen
            if X_val and y_val are not None, it refects whether to evaluate the training data
    """
    # parameters for earlystopping
    best_valid = np.inf
    best_valid_acc = np.inf
    best_valid_epoch = min_epoch

    # Training info
    total_train_loss = []
    total_val_loss = []
    single_step_train_loss = []
    single_step_val_loss = []

    log("Start training the network ...")
    start_time_begin = time.time()
    for epoch in range(n_epoch):
        start_time = time.time()
        loss_ep = 0
        n_step = 0
        log("------------------------- Epoch %d of %d --------------------------" % (epoch + 1, n_epoch))

        for batch in tra_provider.feed_chunk():

            X_train_a, y_train_a = batch
            X_train_a = K.cast_to_floatx(X_train_a)
            y_train_a = K.cast_to_floatx(y_train_a)

            feed_dict = {x: X_train_a, y_: y_train_a}
            #feed_dict.update(network.all_drop)  # enable noise layers
            loss, _ = sess.run([cost, train_op], feed_dict=feed_dict)
            loss_ep += loss
            n_step += 1
            #print("    batch {0:d}".format(n_step))
            #log(tf.trainable_variables())

            """
            for v in tf.trainable_variables():
                if v.name == 'conv2d_1/kernel:0':
                    value = sess.run(v)
                    print(value)
                    break
            """

            #for k, v in zip(variables_names, values):
            #   print(k, v)
        loss_ep = loss_ep / n_step
        log('loss_ep: %f' % loss_ep)

        if epoch >= 0 or (epoch + 1) % print_freq == 0:
            # evaluate the val error at each epoch.
            if val_provider is not None:
                log("Epoch %d of %d took %fs" % (epoch + 1, n_epoch, time.time() - start_time))
                log("Validation...")
                train_loss, train_acc, n_batch_train = 0, 0, 0
                for batch in tra_provider.feed_chunk():
                    X_train_a, y_train_a = batch
                    #dp_dict = dict_to_one(network.all_drop)  # disable noise layers
                    feed_dict = {x: X_train_a, y_: y_train_a}
                    #feed_dict.update(dp_dict)
                    if acc is not None:
                        err, ac = sess.run([cost, acc], feed_dict=feed_dict)
                        train_acc += ac
                    else:
                        err = sess.run(cost, feed_dict=feed_dict)
                    train_loss += err
                    n_batch_train += 1
                    single_step_train_loss.append(err)
                total_train_loss.append(train_loss/n_batch_train)
                log("   train loss/n_batch_train: %f" % (train_loss / n_batch_train))
                log("   train loss: %f, n_batch_train: %d" % (train_loss, n_batch_train))

                if acc is not None:
                    log("   train acc: %f" % (train_acc / n_batch_train))

                val_loss, val_acc, n_batch_val = 0, 0, 0

                for batch in val_provider.feed_chunk():
                    X_val_a, y_val_a = batch
                    #dp_dict = dict_to_one(network.all_drop)  # disable noise layers
                    feed_dict = {x: X_val_a, y_: y_val_a}
                    #feed_dict.update(dp_dict)
                    if acc is not None:
                        err, ac = sess.run([cost, acc], feed_dict=feed_dict)
                        val_acc += ac
                    else:
                        err = sess.run(cost, feed_dict=feed_dict)
                    val_loss += err
                    n_batch_val += 1
                    single_step_val_loss.append(err)
                log("    val loss: %f" % (val_loss / n_batch_val))
                total_val_loss.append(val_loss/n_batch_val)
                if acc is not None:
                    log("   val acc: %f" % (val_acc / n_batch_val))
            else:
                log('no validation')
                log("Epoch %d of %d took %fs, loss %f" % (epoch + 1, n_epoch, time.time() - start_time, loss_ep))

        if earlystopping:
            if epoch >= min_epoch:
                log("Evaluate earlystopping parameters...")
                current_valid = val_loss / n_batch_val
                current_valid_acc = val_acc / n_batch_val
                current_epoch = epoch
                current_train_loss = train_loss / n_batch_train
                current_train_acc = train_acc / n_batch_train
                log('    Current valid loss was {:.6f}, acc was {:.6f}, '
                    'train loss was {:.6f}, acc was {:.6f} at epoch {}.'
                    .format(current_valid, current_valid_acc, current_train_loss, current_train_acc, current_epoch+1))
                if current_valid < best_valid:
                    best_valid = current_valid
                    best_valid_acc = current_valid_acc
                    best_valid_epoch = current_epoch

                    # save the model parameters
                    modelsaver(network=network, path=save_path, epoch_identifier=None)
                    log('Best valid loss was {:.6f} and acc {:.6f} at epoch {}.'.format(
                          best_valid, best_valid_acc, best_valid_epoch+1))
                elif best_valid_epoch + patience < current_epoch:
                    log('Early stopping.')
                    log('Best valid loss was {:.6f} and acc {:.6f} at epoch {}.'.format(
                          best_valid, best_valid_acc, best_valid_epoch+1))
                    break

        else:
            current_val_loss = val_loss / n_batch_val
            current_val_acc = val_acc / n_batch_val
            current_epoch = epoch
            current_train_loss = train_loss / n_batch_train
            current_train_acc = train_acc / n_batch_train
            log('    Current valid loss was {:.6f}, acc was {:.6f}, train loss was {:.6f}, acc was {:.6f} at epoch {}.'
                .format(current_val_loss, current_val_acc, current_train_loss, current_train_acc, current_epoch+1))

            #log(save_model > 0, epoch % save_model == 0, epoch/save_model > 0)
            if save_model > 0 and epoch % save_model == 0:
                if epoch_identifier:
                    modelsaver(network=network, path=save_path, epoch_identifier=epoch+1)
                else:
                    modelsaver(network=network, path=save_path, epoch_identifier=None)

    if not earlystopping:
        if save_model == -1:
            modelsaver(network=network, path=save_path, epoch_identifier=None)

    log("Total training time: %fs" % (time.time() - start_time_begin))
    return total_train_loss, total_val_loss, single_step_train_loss, single_step_val_loss


def custompredictX(sess,
                  network,
                  output_provider,
                  x,
                  fragment_size=1000,
                  output_length=1,
                  y_op=None,
                  out_kwag=None):
    """
        Return the predict results of given non time-series network.

        Parameters
        ----------
        sess : TensorFlow session
            sess = tf.InteractiveSession()
        network : a TensorLayer layer
            the network will be trained
        x : placeholder
            the input
        y_op : placeholder
    """

    if y_op is None:
        y_op = network.outputs

    output_container = []
    banum = 0

    for X_out in output_provider.feed(out_kwag['inputs']):
        #log(banum)
        #banum += 1

        feed_dict = {x: X_out,}
        output = sess.run(y_op, feed_dict=feed_dict)
        output_array = np.array(output[0]).reshape(-1, output_length)
        output_container.append(output_array)

    return np.vstack(output_container)

In [8]:

# from sklearn.metrics import confusion_matrix

def get_TP(target, prediction, threshold):
    '''
    compute the  number of true positive

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    assert (target.shape == prediction.shape)

    target = 1 - np.clip(target, threshold, 0) / threshold
    prediction = 1 - np.clip(prediction, threshold, 0) / threshold

    tp_array = np.logical_and(target, prediction) * 1.0
    tp = np.sum(tp_array)

    return tp


def get_FP(target, prediction, threshold):
    '''
    compute the  number of false positive

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    assert (target.shape == prediction.shape)

    target = np.clip(target, threshold, 0) / threshold
    prediction = 1 - np.clip(prediction, threshold, 0) / threshold

    fp_array = np.logical_and(target, prediction) * 1.0
    fp = np.sum(fp_array)

    return fp


def get_FN(target, prediction, threshold):
    '''
    compute the  number of false negtive

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    assert (target.shape == prediction.shape)

    target = 1 - np.clip(target, threshold, 0) / threshold
    prediction = np.clip(prediction, threshold, 0) / threshold

    fn_array = np.logical_and(target, prediction) * 1.0
    fn = np.sum(fn_array)

    return fn


def get_TN(target, prediction, threshold):
    '''
    compute the  number of true negative

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    assert (target.shape == prediction.shape)

    target = np.clip(target, threshold, 0) / threshold
    prediction = np.clip(prediction, threshold, 0) / threshold

    tn_array = np.logical_and(target, prediction) * 1.0
    tn = np.sum(tn_array)

    return tn


def get_recall(target, prediction, threshold):
    '''
    compute the recall rate

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    tp = get_TP(target, prediction, threshold)
    fn = get_FN(target, prediction, threshold)
    log('tp={0}'.format(tp))
    log('fn={0}'.format(fn))
    if tp + fn <= 0.0:
        recall = tp / (tp + fn + 1e-9)
    else:
        recall = tp / (tp + fn)
    return recall


def get_precision(target, prediction, threshold):
    '''
    compute the  precision rate

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    tp = get_TP(target, prediction, threshold)
    fp = get_FP(target, prediction, threshold)
    log('tp={0}'.format(tp))
    log('fp={0}'.format(fp))
    if tp + fp <= 0.0:
        precision = tp / (tp + fp + 1e-9)
    else:
        precision = tp / (tp + fp)
    return precision


def get_F1(target, prediction, threshold):
    '''
    compute the  F1 score

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    recall = get_recall(target, prediction, threshold)
    log(recall)
    precision = get_precision(target, prediction, threshold)
    log(precision)
    if precision == 0.0 or recall == 0.0:
        f1 = 0.0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    return f1


def get_accuracy(target, prediction, threshold):
    '''
    compute the accuracy rate

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    threshold: float
    '''

    tp = get_TP(target, prediction, threshold)
    tn = get_TN(target, prediction, threshold)

    accuracy = (tp + tn) / target.size

    return accuracy


def get_relative_error(target, prediction):
    '''
    compute the  relative_error

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    '''

    assert (target.shape == prediction.shape)

    return np.mean(np.nan_to_num(np.abs(target - prediction) / np.maximum(target, prediction)))


def get_abs_error(target, prediction):
    '''
    compute the  absolute_error

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    '''

    assert (target.shape == prediction.shape)

    data = np.abs(target - prediction)
    mean, std, min_v, max_v, quartile1, median, quartile2 = get_statistics(data)

    return mean, std, min_v, max_v, quartile1, median, quartile2, data


def get_nde(target, prediction):
    '''
    compute the  normalized disaggregation error

    Parameters:
    ----------------
    target: the groud truth , np.array
    prediction: the prediction, np.array
    '''

    return np.sum((target - prediction) ** 2) / np.sum((target ** 2))


def get_sae(target, prediction, sample_second):
    '''
    compute the signal aggregate error
    sae = |\hat(r)-r|/r where r is the ground truth total energy;
    \hat(r) is the predicted total energy.
    '''
    r = np.sum(target * sample_second * 1.0 / 3600.0)
    rhat = np.sum(prediction * sample_second * 1.0 / 3600.0)

    sae = np.abs(r - rhat) / np.abs(r)

    return sae

def get_Epd(target, prediction, sample_second):
    '''
    Energy per day
    - calculate energy of a day for both ground truth and prediction
    - sum all the energies
    - divide by the number of days
    '''

    day = int(24.0 * 3600 / sample_second)
    gt_en_days = []
    pred_en_days = []

    for start in range(0, int(len(target)-day), int(day)):
        gt_en_days.append(np.sum(target[start:start+day]*sample_second)/3600)
        pred_en_days.append(np.sum(prediction[start:start+day]*sample_second)/3600)

    Epd = np.sum(np.abs(np.array(gt_en_days)-np.array(pred_en_days)))/(len(target)/day)

    return Epd


def get_statistics(data):

    mean = np.mean(data)
    std = np.std(data)
    min_v = np.sort(data)[0]
    max_v = np.sort(data)[-1]

    quartile1 = np.percentile(data, 25)
    median = np.percentile(data, 50)
    quartile2 = np.percentile(data, 75)

    return mean, std, min_v, max_v, quartile1, median, quartile2

In [ ]:
import os
import tensorflow as tf
from keras.layers import Input
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse

def remove_space(string):
    return string.replace(" ","")

def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')
    

"""
def get_arguments():
    parser = argparse.ArgumentParser(description='Train a neural network\
                                     for energy disaggregation - \
                                     network input = mains window; \
                                     network target = the states of \
                                     the target appliance.')
    parser.add_argument('--appliance_name',
                        type=remove_space,
                        default='kettle',
                        help='the name of target appliance')
    parser.add_argument('--datadir',
                        type=str,
                        default='/media/michele/Dati/myREFIT/',
                        help='this is the directory of the training samples')
    parser.add_argument('--pretrainedmodel_dir',
                        type=str,
                        default='./pretrained_model',
                        help='this is the directory of the pre-trained models')
    parser.add_argument('--save_dir',
                        type=str,
                        default='./models',
                        help='this is the directory to save the trained models')
    parser.add_argument('--batchsize',
                        type=int,
                        default=1000,
                        help='The batch size of training examples')
    parser.add_argument('--n_epoch',
                        type=int,
                        default=5,
                        help='The number of epochs.')
    parser.add_argument('--save_model',
                        type=int,
                        default=-1,
                        help='Save the learnt model:\
                        0 -- not to save the learnt model parameters;\
                        n (n>0) -- to save the model params every n steps;\
                        -1 -- only save the learnt model params\
                        at the end of training.')
    parser.add_argument('--dense_layers',
                        type=int,
                        default=1,
                        help=':\
                                1 -- One dense layers (default Seq2point);\
                                2 -- Two dense layers;\
                                3 -- Three dense layers.')
    parser.add_argument("--transfer_model", type=str2bool,
                        default=False,
                        help="True: using entire pre-trained model.\
                             False: retrain the entire pre-trained model;\
                             This will override the 'transfer_cnn' and 'cnn' parameters;\
                             The appliance_name parameter will use to retrieve \
                             the entire pre-trained model of that appliance.")
    parser.add_argument("--transfer_cnn", type=str2bool,
                        default=False,
                        help="True: using a pre-trained CNN\
                              False: not using a pre-trained CNN.")
    parser.add_argument('--cnn',
                        type=str,
                        default='kettle',
                        help='The CNN trained by which appliance to load (pretrained model).')
    parser.add_argument('--gpus',
                        type=int,
                        default=-1,
                        help='Number of GPUs to use:\
                            n -- number of GPUs the system should use;\
                            -1 -- do not use any GPU.')
    parser.add_argument('--crop_dataset',
                        type=int,
                        default=10000,
                        help='for debugging porpose should be helpful to crop the training dataset size')
    parser.add_argument('--ram',
                        type=int,
                        default=5*10**5,
                        help='Maximum number of rows of csv dataset can handle without loading in chunks')
    return parser.parse_args()
"""


class get_arguments:
    def __init__(self):
        self.data_dir = 'UKDALE/'
        self.appliance_name = 'kettle'
        self.aggregate_mean = AGG_MEAN
        self.aggregate_std = AGG_STD
        self.save_path = SAVE_PATH  # Added to fix AttributeError
        self.save_dir = 'transfer_learned_models/'  # Added to fix AttributeError
        self.pretrainedmodel_dir = 'saved_models/kettle_best_model'
        self.batchsize = 1000
        self.n_epoch = 5
        self.save_model = -1
        self.dense_layers = 1
        self.transfer_model = False
        self.transfer_cnn = False
        self.cnn = 'kettle'
        self.gpus = 1
        self.crop_dataset = 10000
        self.ram = 5*10**5



args = get_arguments()
log('Arguments: ')
log(args)

# some constant parameters
CHUNK_SIZE = 5*10**6

# Reset the default graph and start the session for training a network
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.InteractiveSession()

# the appliance to train on
appliance_name = args.appliance_name

# path for training data
training_path = args.data_dir + appliance_name + '/' + appliance_name + '_training_' + '.csv'
log('Training dataset: ' + training_path)

# Looking for the validation set
for filename in os.listdir(args.data_dir + appliance_name):
    if "validation" in filename:
        val_filename = filename
        log(val_filename)

# path for validation data
validation_path = args.data_dir + appliance_name + '/' + val_filename
log('Validation dataset: ' + validation_path)


# offset parameter from window length
offset = int(0.5*(params_appliance[args.appliance_name]['windowlength']-1.0))

# Defining object for training set loading and windowing provider (DataProvider.py)
tra_provider = ChunkDoubleSourceSlider2(filename=training_path,
                                        batchsize=args.batchsize,
                                        chunksize = CHUNK_SIZE,
                                        crop=args.crop_dataset,
                                        shuffle=True,
                                        offset=offset,
                                        header=0,
                                        ram_threshold=args.ram)

# Defining object for validation set loading and windowing provider (DataProvider.py)
val_provider = ChunkDoubleSourceSlider2(filename=validation_path,
                                        batchsize=args.batchsize,
                                        chunksize=CHUNK_SIZE,
                                        crop=args.crop_dataset,
                                        shuffle=False,
                                        offset=offset,
                                        header=0,
                                        ram_threshold=args.ram)

# TensorFlow placeholders
x = tf.compat.v1.placeholder(tf.float32,
                   shape=[None, params_appliance[args.appliance_name]['windowlength']],
                   name='x')

y_ = tf.compat.v1.placeholder(tf.float32,
                    shape=[None, 1],
                    name='y_')

# -------------------------------- Keras Network - from model.py -----------------------------------------
inp = Input(tensor=x)
model, cnn_check_weights = get_model(args.appliance_name,
                                     inp,
                                     params_appliance[args.appliance_name]['windowlength'],
                                     transfer_dense=args.transfer_model,
                                     transfer_cnn=args.transfer_cnn,
                                     cnn=args.cnn,
                                     pretrainedmodel_dir=args.pretrainedmodel_dir)
y = model.outputs
# -------------------------------------------------------------------------------------------------------

# cost function
cost = tf.reduce_mean(tf.reduce_mean(tf.math.squared_difference(y, y_), 1))

# model's weights to be trained
train_params = tf.compat.v1.trainable_variables()
log("All network parameters: ")
log([v.name for v in train_params])
# if transfer learning is selected, just the dense layer will be trained
if not args.transfer_model and args.transfer_cnn:
    parameters = 10
else:
    parameters = 0
log("Trainable parameters:")
log([v.name for v in train_params[parameters:]])

# Training hyper parameters
train_op = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001,
                                            beta1=0.9,
                                            beta2=0.999,
                                            epsilon=1e-08,
                                            use_locking=False).minimize(cost,
                                                                        var_list=train_params[parameters:]
                                                                        )

# Initialize all variables in the graph (must be after model is built)
sess.run(tf.compat.v1.global_variables_initializer())


log('TensorFlow Session starting...')

# TensorBoard summary (graph)
tf.compat.v1.summary.scalar('cost', cost)
merged_summary = tf.compat.v1.summary.merge_all()
writer = tf.compat.v1.summary.FileWriter('./tensorboard_test')
writer.add_graph(sess.graph)
log('TensorBoard infos in ./tensorboard_test')

# Save path depending on the training behaviour
if not args.transfer_model and args.transfer_cnn:
    save_path = args.save_dir+'/cnn_s2p_' + appliance_name + '_transf_' + args.cnn + '_pointnet_model'
else:
    save_path = args.save_dir+'/cnn_s2p_' + appliance_name + '_pointnet_model'
    
if not os.path.exists(save_path):
        os.makedirs(save_path)

# Replace custom training function with standard Keras training loop
# You may need to adapt this to your data provider's API

train_loss, val_loss, step_train_loss, step_val_loss = customfit(sess=sess,
                                                                    network=model,
                                                                    cost=cost,
                                                                    train_op=train_op,
                                                                    tra_provider=tra_provider,
                                                                    x=x,
                                                                    y_=y_,
                                                                    acc=None,
                                                                    n_epoch=args.n_epoch,
                                                                    print_freq=1,
                                                                    val_provider=val_provider,
                                                                    save_model=args.save_model,
                                                                    save_path=save_path,
                                                                    epoch_identifier=None,
                                                                    earlystopping=True,
                                                                    min_epoch=1,
                                                                    patience=1)

# Following are training info
"""
log('train loss: ' + str(train_loss))
log('val loss: ' + str(val_loss))
infos = pd.DataFrame(data={'train_loss': step_train_loss,
                           #'val_loss': step_val_loss
                           })
infos.to_csv('./training_infos-{:}-{:}-{:}.csv'.format(appliance_name, args.transfer, args.cnn))
log('training infos in .csv file')
"""

# This check that the CNN is the same of the beginning
if not args.transfer_model and args.transfer_cnn:
    log('Transfer learning check ...')
    session = K.get_session()
    for v in tf.trainable_variables():
        if v.name == 'conv2d_1/kernel:0':
            value = session.run(v)
            vl = np.array(value).flatten()
            c1 = np.array(cnn_check_weights).flatten()
            if False in vl == c1:
                log('Transfer check --- ERROR ---')
            else:
                log('Transfer check --- OK ---')


sess.close()






2025-06-20 03:48:32,050 [INFO ]  Arguments: 
2025-06-20 03:48:32,069 [INFO ]  <__main__.get_arguments object at 0x00000286CE587760>
2025-06-20 03:48:32,074 [INFO ]  Training dataset: UKDALE/kettle/kettle_training_.csv
2025-06-20 03:48:32,075 [INFO ]  kettle_validation_.csv
2025-06-20 03:48:32,077 [INFO ]  Validation dataset: UKDALE/kettle/kettle_validation_.csv
2025-06-20 03:48:32,558 [INFO ]  Standard training...
2025-06-20 03:48:32,558 [INFO ]  ...creating a new model.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 599)]             0         
                                                                 
 reshape (Reshape)           (None, 1, 599, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 1, 599, 30)        330       
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 599, 30)        7230      
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 599, 40)        7240      
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 599, 50)        10050     
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 599, 50)        12550 

2025-06-20 03:48:32,733 [INFO ]  All network parameters: 
2025-06-20 03:48:32,737 [INFO ]  ['conv2d/kernel:0', 'conv2d/bias:0', 'conv2d_1/kernel:0', 'conv2d_1/bias:0', 'conv2d_2/kernel:0', 'conv2d_2/bias:0', 'conv2d_3/kernel:0', 'conv2d_3/bias:0', 'conv2d_4/kernel:0', 'conv2d_4/bias:0', 'dense/kernel:0', 'dense/bias:0', 'output/kernel:0', 'output/bias:0']
2025-06-20 03:48:32,740 [INFO ]  Trainable parameters:
2025-06-20 03:48:32,742 [INFO ]  ['conv2d/kernel:0', 'conv2d/bias:0', 'conv2d_1/kernel:0', 'conv2d_1/bias:0', 'conv2d_2/kernel:0', 'conv2d_2/bias:0', 'conv2d_3/kernel:0', 'conv2d_3/bias:0', 'conv2d_4/kernel:0', 'conv2d_4/bias:0', 'dense/kernel:0', 'dense/bias:0', 'output/kernel:0', 'output/bias:0']
2025-06-20 03:48:34,297 [INFO ]  TensorFlow Session starting...
2025-06-20 03:48:35,087 [INFO ]  TensorBoard infos in ./tensorboard_test
2025-06-20 03:48:35,292 [INFO ]  Start training the network ...
2025-06-20 03:48:35,330 [INFO ]  ------------------------- Epoch 1 of 5 --------------

: 